In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# family = "AppleGothic"
family = "Malgun Gothic"

plt.rc('font', family=family)

In [ ]:
df = pd.read_csv('data/서울특별시/apartment_df.csv', index_col=0)
df

In [ ]:
corr_df = df.corr()
corr_df

In [ ]:
sns.heatmap(corr_df, annot=True, cmap="RdYlBu_r", vmin=-1, vmax=1)

In [ ]:
sns.clustermap(corr_df, annot=True, cmap="RdYlBu_r", vmin=-1, vmax=1)

In [ ]:
df.head(1)

In [ ]:
sns.boxplot(data=df, sym='r+', y='transaction_real_price')

In [ ]:
df.sort_values('transaction_real_price', ascending=False)

In [ ]:
location = pd.read_csv('data/동별좌표.csv')
location

In [ ]:
def search_location(dong):
    try : 
        dong_data = location[location['동'] == dong].iloc[0]
        return (dong_data['loc'], dong_data['lat'])
    except :
        print(dong)
        return (np.nan, np.nan)

In [ ]:
len(np.unique(df['dong']))

In [ ]:
price_for_dong = df.groupby('dong').mean()[['transaction_real_price']]
price_for_dong = price_for_dong.reset_index()
price_for_dong

In [ ]:
count_for_dong = df.groupby('dong').count()[['transaction_real_price']]
count_for_dong = count_for_dong.reset_index()
count_for_dong

In [ ]:
merge_df = pd.merge(price_for_dong, count_for_dong, on='dong')
merge_df.columns = ['dong', 'mean_price', 'count']
merge_df

In [ ]:
merge_df[['loc', 'lat']] = merge_df['dong'].apply(search_location).apply(pd.Series)
merge_df

In [ ]:
sns.scatterplot(data=merge_df, x='lat', y='loc', hue='dong', size='mean_price', sizes=(10, 200), legend=False)

In [ ]:
import plotly.express as px
fig = px.scatter(merge_df, x='lat', y='loc', color='dong', size='mean_price')
fig.show()

In [ ]:
sns.scatterplot(data=merge_df, x='lat', y='loc', hue='dong', size='count', sizes=(10, 200), legend=False)

In [ ]:
import plotly.express as px
fig = px.scatter(merge_df, x='lat', y='loc', color='dong', size='count')
fig.show()

In [ ]:
# !pip install folium

In [ ]:
import folium

m = folium.Map(location=[37.5, 127], zoom_start=11)
m

In [ ]:
for idx, row in merge_df.iterrows():
    location = [row['loc'], row['lat']]
    folium.CircleMarker(
        location=location,
        popup=row['dong'] + str(int(row['mean_price'])),
        radius=row['mean_price']/10000,
        fill=True,
    ).add_to(m)
m   

In [ ]:
for idx, row in merge_df.iterrows():
    location = [row['loc'], row['lat']]
    folium.CircleMarker(
        location=location,
        popup=row['dong'] + str(int(row['count'])),
        radius=row['count']/1000,
        color='red',
        fill=True,
    ).add_to(m)
m   

#### 의미있는 컬럼을 더 만들어보자

In [ ]:
df['transaction_year'] = (df['transaction_year_month']/100).astype(int)
df['transaction_month'] = df['transaction_year_month']%100
df

In [ ]:
df['transaction_date_first'] = df['transaction_date'].str.split('~').str[0].astype(int)
df

In [ ]:
df['real_price_per_m2'] = df['transaction_real_price'] / df['exclusive_use_area']
df

In [ ]:
df.groupby('dong').mean()